In [8]:
def solved(size):
    puzzle = []
    for r in range(size):
        row = []
        for c in range(size):
            row.append((r*size)+c+1)
        puzzle.append(row)
    puzzle[size-1][size-1] = 0
    return puzzle
            

In [9]:
print(solved(3))


[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
